In [1]:
from __future__ import division
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
df_total=pd.read_csv('fda.tsv', sep="\t")
df=df_total[['primaryid','drug_seq','caseid','indi_pt','start_dt','end_dt','dur','dur_cod','drugname','route','dose_vbm','cum_dose_chr','cum_dose_unit','rechal','dose_amt','dose_unit','dose_form','dose_freq','age','age_cod','gndr_cod','wt','wt_cod','reporter_country','dt.1','lt','ho','ds','ca','ri','ot','reactions']]
df.rename(columns={'reactions': 'pt', 'dt.1': 'de'}, inplace=True)
df.head()

,primaryid,drug_seq,caseid,indi_pt,start_dt,end_dt,dur,dur_cod,drugname,route,...,wt_cod,reporter_country,de,lt,ho,ds,ca,ri,ot,pt
0,100033001,1,10003300.0,Osteoporosis,20130913.0,NaN,NaN,NaN,BONIVA,INTRAVENOUS,...,NaN,US,NaN,NaN,NaN,14Q1,NaN,NaN,NaN,"Arthralgia,Headache"
1,100033061,1,10003306.0,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough
2,100033062,1,10003306.0,Product used for unknown indication,20111014.0,NaN,NaN,NaN,LETAIRIS,Oral,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough
3,100033072,2,10003307.0,Connective tissue disorder,NaN,NaN,NaN,NaN,LETAIRIS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhinorrhoea
4,100033073,2,10003307.0,Connective tissue disorder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhinorrhoea


In [3]:
# Clearning Part
df['pt']=df['pt'].fillna('')
df['age']=df['age'].fillna('')
df['wt']=df['wt'].fillna('')
df['gndr_cod']=df['gndr_cod'].fillna('')
df['reporter_country']=df['reporter_country'].fillna('')
#df=pd.concat([df[col].astype(str).apply(lambda x: x.lower()) for col in df.columns], axis=1)
df.loc[df['de'].notnull(), 'de'] = 'death'
df.loc[df['lt'].notnull(), 'lt'] = 'life_threat'
df.loc[df['ho'].notnull(), 'ho'] = 'hospitalization'
df.loc[df['ds'].notnull(), 'ds'] = 'disability'
df.loc[df['ca'].notnull(), 'ca'] = 'congenital_anomaly'
df.loc[df['ri'].notnull(), 'ri'] = 'required_intervention'
df.loc[df['ot'].notnull(), 'ot'] = 'other_serious'
df.head()

,primaryid,drug_seq,caseid,indi_pt,start_dt,end_dt,dur,dur_cod,drugname,route,...,wt_cod,reporter_country,de,lt,ho,ds,ca,ri,ot,pt
0,100033001,1,10003300.0,Osteoporosis,20130913.0,NaN,NaN,NaN,BONIVA,INTRAVENOUS,...,NaN,US,NaN,NaN,NaN,disability,NaN,NaN,NaN,"Arthralgia,Headache"
1,100033061,1,10003306.0,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough
2,100033062,1,10003306.0,Product used for unknown indication,20111014.0,NaN,NaN,NaN,LETAIRIS,Oral,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough
3,100033072,2,10003307.0,Connective tissue disorder,NaN,NaN,NaN,NaN,LETAIRIS,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhinorrhoea
4,100033073,2,10003307.0,Connective tissue disorder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhinorrhoea


In [4]:
df.loc[df['age_cod']=='85', 'age'] = 85
df = df[df.age_cod != 'DEC'].reset_index(drop=True)
df['age']=df['age'].apply(pd.to_numeric, errors='coerce')
df['age'] = np.where(df['age_cod'] == 'MON', df['age'] * 1/12, df['age'])
df['age'] = np.where(df['age_cod'] == 'DY', df['age'] * 1/365, df['age'])
df['age'] = np.where(df['age_cod'] == 'WK', df['age'] * 1/52, df['age'])
df['age'] = np.where(df['age_cod'] == 'MIN', df['age'] * 1/525600, df['age'])
df=df.drop(['age_cod'], axis=1) #Droping age code column

In [5]:
df['wt']=df['wt'].apply(pd.to_numeric, errors='coerce')
df['wt'] = np.where(df['wt_cod'] == 'LBS', df['wt'] * 0.453592, df['wt'])
df['wt'] = np.where(df['wt_cod'] == 'GMS', df['wt'] * 0.001, df['wt'])
df = df[df.wt_cod != '20050308']
df = df[df.wt_cod != '20060127']
df = df[df.wt_cod != '20060330']
df = df[df.wt_cod != '20080808']
df = df[df.wt_cod != 'LG']
df = df[df.wt_cod != 'YEARS']
df = df[df.wt_cod != 'L']
df = df[df.wt_cod != 'Y']
df = df[df.wt_cod != 'M']
df=df.drop(['wt_cod'], axis=1) #Droping weight code column

In [ ]:
# Old Method
#df['age']=df['age'].apply(pd.to_numeric, errors='coerce')
#df['wt']=df['wt'].apply(pd.to_numeric, errors='coerce')
#indx=df.index[df['age_cod']=='MON']
#for i in indx:
#    df.iloc[i,18]=float(df.iloc[i,18])*(1/12)
#indx=df.index[df['age_cod']=='DY']
#for i in indx:
#    df['age'][i]=float(df['age'][i])*(1/365)
#
#indx=df.index[df['age_cod']=='WK']
#for i in indx:
#    df['age'][i]=float(df['age'][i])*(1/52)
#
#indx=df.index[df['age_cod']=='MIN']
#for i in indx:
#    df['age'][i]=float(df['age'][i])*(1/525600)
#df=df.drop(['age_cod'], axis=1)

In [ ]:
# Old Method
#indx=df.index[df['wt_cod']=='LBS']
#for i in indx:
#    df['wt'][i]=float(df['wt'][i])*(0.453592)
#
#indx=df.index[df['wt_cod']=='GMS']
#for i in indx:
#    df['wt'][i]=float(df['wt'][i])*(0.001)

#df = df[df.wt_cod != '20050308']
#df = df[df.wt_cod != '20060127']
#df = df[df.wt_cod != '20060330']
#df = df[df.wt_cod != '20080808']
#df = df[df.wt_cod != 'LG']
#df = df[df.wt_cod != 'YEARS']
#df = df[df.wt_cod != 'L']
#df = df[df.wt_cod != 'Y']
#df = df[df.wt_cod != 'M']
#df=df.drop(['wt_cod'], axis=1) #Droping weight code column

In [6]:
df['gndr_cod']=df['gndr_cod'].str.replace('NS','') # Data Cleaning
df['gndr_cod']=df['gndr_cod'].str.replace('UNK','') # Data Cleaning
df = df[df.gndr_cod != 'Y']
df = df[df.gndr_cod != 'YR']
df = df[df.gndr_cod != 'N']
df['rechal']=df['rechal'].str.replace('Y','positive') # Data Cleaning
df['rechal']=df['rechal'].str.replace('D','') # Data Cleaning
df['rechal']=df['rechal'].str.replace('U','') # Data Cleaning
df['rechal']=df['rechal'].str.replace('1030093','') # Data Cleaning
df['rechal']=df['rechal'].str.replace('N','negative') # Data Cleaning
df.head()

,primaryid,drug_seq,caseid,indi_pt,start_dt,end_dt,dur,dur_cod,drugname,route,...,wt,reporter_country,de,lt,ho,ds,ca,ri,ot,pt
0,100033001,1,10003300.0,Osteoporosis,20130913.0,NaN,NaN,NaN,BONIVA,INTRAVENOUS,...,NaN,US,NaN,NaN,NaN,disability,NaN,NaN,NaN,"Arthralgia,Headache"
1,100033061,1,10003306.0,Product used for unknown indication,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough
2,100033062,1,10003306.0,Product used for unknown indication,20111014.0,NaN,NaN,NaN,LETAIRIS,Oral,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cough
3,100033072,2,10003307.0,Connective tissue disorder,NaN,NaN,NaN,NaN,LETAIRIS,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhinorrhoea
4,100033073,2,10003307.0,Connective tissue disorder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhinorrhoea


In [14]:
dictionary = df.groupby(['caseid'])['age', 'gndr_cod', 'wt','reporter_country'].max().reset_index().set_index('caseid').to_dict('index')

In [ ]:
df.groupby(['caseid'])['age', 'gndr_cod', 'wt','reporter_country'].max().reset_index().set_index('caseid')

In [16]:
import json
import io
with open('dict.txt', 'w') as f:
  json.dump(dictionary,f, ensure_ascii=True)

In [ ]:
import time
start = time.time()
for i in range(len(df)):
    cid =  df['caseid'][i]
    df['age'][i] = dictionary[cid]['age']
    df['wt'][i] = dictionary[cid]['wt']
    df['gndr_cod'][i] = dictionary[cid]['gndr_cod']
    df['reporter_country'][i] = dictionary[cid]['reporter_country']
    if i % 1000 == 0:
        now = time.time()
        print i
        print now-start
df.to_csv('fda_clean2.csv', index=False)

In [ ]:
df2=(df.set_index(df.columns.drop('pt',1).tolist()).pt.str.split(',', expand=True).stack().reset_index().rename(columns={0:'pt'}).loc[:, df.columns])
df2.to_csv('fda_cleaned3.csv',index=False)

In [43]:
df=pd.read_csv('fda_clean1.csv')
df.head()

,primaryid,drug_seq,caseid,indi_pt,drugname,age,gndr_cod,wt,reporter_country,de,lt,ho,ds,ca,ri,ot,pt
0,100033001,1,10003300.0,osteoporosis,boniva,77.0,f,NaN,us,NaN,NaN,NaN,disability,NaN,NaN,NaN,"arthralgia,headache"
1,100033061,1,10003306.0,product used for unknown indication,NaN,NaN,f,NaN,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cough
2,100033062,1,10003306.0,product used for unknown indication,letairis,68.0,f,NaN,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cough
3,100033072,2,10003307.0,connective tissue disorder,letairis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rhinorrhoea
4,100033073,2,10003307.0,connective tissue disorder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rhinorrhoea


In [11]:
ss=ddd['caseid'].value_counts().reset_index()
len(ss['caseid']>1)

3131711

In [12]:
ss

,index,caseid
0,12790479.0,314
1,8583598.0,248
2,8037927.0,170
3,12511430.0,167
4,9641232.0,159
5,8526349.0,159
6,10010901.0,159
7,10049751.0,148
8,8817364.0,147
9,10417854.0,145


In [14]:
ls=ss['index']

In [41]:
id_list=ddd[ddd['caseid'].isin(ls)][['caseid','age','wt','gndr_cod','reporter_country']].dropna(thresh=2)['caseid'].unique()

In [47]:
df['wt'].max()!=None

True

In [ ]:
print "How many rows?" , len(df)
for i in id_list:
    dfn=df[df['caseid']==i]
    if dfn['age'].max()!=None:
        age=dfn['age'].max()
        df.loc[df['caseid']==i, 'age'] = age
    if dfn['gndr_cod'].max()!=None:
        gndr=dfn['gndr_cod'].max()
        df.loc[df['caseid']==i, 'gndr_cod'] = gndr
    if dfn['wt'].max()!=None:
        wt=dfn['wt'].max()
        df.loc[df['caseid']==i, 'wt'] = wt
    if dfn['reporter_country'].max()!=None:
        country=dfn['reporter_country'].max()
        df.loc[df['caseid']==i, 'reporter_country'] = country
print "How many rows?" , len(df)
df.to_csv('fda_cleaned.csv',index=False)

In [29]:
dff=ddd[['age','wt','gndr_cod','reporter_country']]
dff.dropna(thresh=4)
#df[pd.isnull(df).any(axis=1)]

,age,wt,gndr_cod,reporter_country
0,77.00,NaN,f,us
2,68.00,NaN,f,us
6,59.00,NaN,f,us
15,81.00,NaN,f,us
19,46.00,65.5,m,us
20,59.81,57.8,f,br
21,59.81,57.8,f,br
22,59.81,57.8,f,br
23,59.81,57.8,f,br
29,71.80,NaN,m,jp


In [34]:
len(dff.dropna(thresh=1))

4775993

In [25]:
dff=ddd[ddd['caseid'].isin(ls)]